In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from sklearn.manifold import TSNE  
from scipy.special import erfinv
from bokeh.palettes import viridis
import cuml

import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-jan-2021/train.csv")
train = train.loc[train["target"] > 5]
X = train.iloc[:,1:15]
Y = train.iloc[:,15]
test = pd.read_csv("../input/tabular-playground-series-jan-2021/test.csv")
test = test.iloc[:,1:15]

Rank Gauss

In [ ]:
def rg(df, e, start, end):
    for i in df.columns[start:end]:
        r = df[i].rank()
        Range = (r/np.max(r)-0.5)*2
        Range = np.clip(Range, a_max = 1-e, a_min = -1+e)
        rg = erfinv(Range)
        df[i] = rg * 2**0.5
    return df

In [ ]:
X = rg(df = X, e = 0.000001, start = 0, end = 14)

In [ ]:
tSNE = cuml.TSNE(n_components=2, perplexity=25, n_neighbors = 80)
dim3 = tSNE.fit_transform(X)
X["tSNE1"] = dim3[:,0]
X["tSNE2"] = dim3[:,1]

In [ ]:
uMAP = cuml.UMAP(n_components=2, n_neighbors=15, min_dist=0.5)
dim3 = uMAP.fit_transform(X)
X["UMAP1"] = dim3[:,0]
X["UMAP2"] = dim3[:,1]
X["target"] = Y

# t-SNE

In [ ]:
fig = px.scatter(X, 
                 x='tSNE1', 
                 y='tSNE2', 
                 color='target',
                 hover_data={'tSNE1': False, 
                             'tSNE2': False,
                             'target': True
                         },
                 opacity=0.8,
                 color_continuous_scale=px.colors.sequential.Magma,
                 title="t-SNE"
                 )

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

fig.show()

# UMAP

In [ ]:
fig = px.scatter(X, 
                 x='UMAP1', 
                 y='UMAP2', 
                 color='target',
                 hover_data={'UMAP1': False, 
                             'UMAP2': False,
                             'target': True
                         },
                 opacity=0.8,
                 color_continuous_scale=px.colors.sequential.Viridis,
                 title="UMAP"
                 )

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

fig.show()